In [18]:
from io import StringIO
import pandas as pd
import requests 
import datetime
from bs4 import BeautifulSoup
import time

In [122]:
url = f'https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1101&SYEAR=2017&SSEASON=3&REPORT_ID=C'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(url, headers=headers)
response.encoding='big5'
dfs = pd.read_html(StringIO(response.text))
data = dfs[2].transpose()
data = data.reset_index().drop(columns='level_1')
data.columns = data.iloc[0]
data = data.drop(data.index[0])
data = data.rename(columns={'會計項目':'Timestamp'})
data = data.iloc[0]
new_timestamp = data['Timestamp'].replace('年第','Q').replace('季','')
data['Timestamp'] = new_timestamp
index_to_terms_dict = {index: term for index, term in enumerate(data)}


0
Timestamp           2017Q3
營業收入                   NaN
營業收入合計          23058257.0
營業成本                   NaN
營業成本合計          18801014.0
                   ...    
繼續營業單位淨利（淨損）          0.53
基本每股盈餘合計              0.53
稀釋每股盈餘                 NaN
繼續營業單位淨利（淨損）          0.53
稀釋每股盈餘合計              0.53
Name: 1, Length: 61, dtype: object

In [124]:
index_to_terms_dict = {index: term for index, term in enumerate(data)}
index_to_terms_dict

{0: '2017Q3',
 1: nan,
 2: 23058257.0,
 3: nan,
 4: 18801014.0,
 5: 4257243.0,
 6: 4257243.0,
 7: nan,
 8: nan,
 9: 188481.0,
 10: nan,
 11: 815035.0,
 12: nan,
 13: 7878.0,
 14: 1011394.0,
 15: 3245849.0,
 16: nan,
 17: nan,
 18: nan,
 19: 58830.0,
 20: 501087.0,
 21: 57942.0,
 22: 617859.0,
 23: nan,
 24: 734.0,
 25: 242161.0,
 26: 6833.0,
 27: -3874.0,
 28: -244386.0,
 29: nan,
 30: 477503.0,
 31: nan,
 32: 330407.0,
 33: 226377.0,
 34: 3472226.0,
 35: nan,
 36: 785176.0,
 37: 2687050.0,
 38: 2687050.0,
 39: nan,
 40: nan,
 41: 2125865.0,
 42: 2709169.0,
 43: 0.0,
 44: nan,
 45: 91716.0,
 46: 4926750.0,
 47: 4926750.0,
 48: 7613800.0,
 49: nan,
 50: 1966857.0,
 51: 720193.0,
 52: nan,
 53: 6048278.0,
 54: 1565522.0,
 55: nan,
 56: 0.53,
 57: 0.53,
 58: nan,
 59: 0.53,
 60: 0.53}

In [121]:
# from mongo import Mongo

# financial_terms_dict = {term: index for index, term in enumerate(data.index)}
# financial_terms_dict
# repo = Mongo(db='trading_bot', collection='incomestatement_index')
# repo.send_document(financial_terms_dict)

In [67]:
new_index = []
for i in data.index:
    i = i.replace('年第','Q').replace('季','')
    new_index.append(i)


In [68]:
new_index

['2017Q3', '2016Q3', '2017年01月01日至2017年09月30日', '2016年01月01日至2016年09月30日']

In [14]:
res.encoding = 'big5'
dfs = pd.read_html(StringIO(res.text))
balance_sheet = dfs[1]
balance_sheet.columns = balance_sheet.columns.droplevel(1)

balance_sheet_index=balance_sheet.iloc[:, 0]
balance_sheet = balance_sheet.iloc[:, 1]
a = pd.concat([date_stock_id_value, balance_sheet])


NameError: name 'date_stock_id_value' is not defined

In [ ]:
date_stock_id_index = pd.Series(['Timestamp', 'stock_id'])

In [ ]:

import datetime as dt 
date = dt.datetime(2022, 2, 1)
date.quarter

In [ ]:
date_stock_id_value = pd.Series(['1101', pd.Timestamp('2021-09-30').strftime('%Y-%m-%d')])
b= pd.concat([date_stock_id_index, balance_sheet_index] )

In [ ]:
pd.concat([b,a], axis=1)

In [ ]:
IS_scraper = IncomeStatementsScraper() #實際在抓歷史資料時，用默認值即可
IS_1101 = IS_scraper.get_income_statements_hst(1101) # 3372 is a good example showing 2 kind of 財報（合併 or 個別）

In [66]:
#!/usr/bin/env python
# coding: utf-8

import datetime
import random
import time
import urllib.request
import requests
from bs4 import BeautifulSoup
import pandas as pd

class IncomeStatementsScraper:
    
    def __init__(self, start_year=2013, end_year=datetime.datetime.now().year):
        if start_year < 2013 or start_year < 0:
            raise ValueError("start_year 必須大於等於 2013 並且不能為負數")
        if end_year < start_year or end_year < 0:
            raise ValueError("end_year 必須大於等於 start_year 並且不能為負數")
        if end_year > datetime.datetime.now().year:
            print('end_year 強制設定為今年')
            self.end_year = datetime.datetime.now().year
            self.year_now = self.end_year
        else:
            self.end_year = end_year
            self.year_now = datetime.datetime.now().year
        
        self.start_year = start_year
        self.dfs = []
        
        self.year_range = list(range(self.start_year, self.end_year+1))
        self.season_range = list(range(1, 5))        
    
    # func 1 
    def get_income_statements_hst(self, stock_id=2330): # need to choose a stock id
        self.stock_id = stock_id

        # Calculate the newInfo_days for 財報發佈日 
        # func 2 is related
        mar_newInfo_day_B = self.get_newInfo_day(1 + self.end_year, 3, 31)
        mar_newInfo_day = self.get_newInfo_day(self.end_year, 3, 31)
        may_newInfo_day = self.get_newInfo_day(self.end_year, 5, 15)
        aug_newInfo_day = self.get_newInfo_day(self.end_year, 8, 14)
        nov_newInfo_day = self.get_newInfo_day(self.end_year, 11, 14)

        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

        # Loop through each year and quarter to retrieve income statements
        for year in self.year_range:
            print(f'{year}年開爬!!')
            for season in self.season_range:                
                if year == self.end_year:
                    if (year == self.year_now - 1) & (season == 4):
                            if datetime.datetime.now() < mar_newInfo_day_B:
                                print('已爬至歷史最新資料，正常結束。 產出主鍵：年、季度。')
                                self.PK = self.PK_year_season(self.dfs) # func 6 is related
                                return self.dfs
                    
                    elif year == self.year_now:
                        checkTool = (mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day)
                        for i, newInfo_day in enumerate(checkTool):
                            if (datetime.datetime.now() < newInfo_day) & (season >= i):
                                print('已爬至歷史最新資料，正常結束。 產出主鍵：年、季度。')
                                self.PK = self.PK_year_season(self.dfs) # func 6 is related
                                return self.dfs
                ### main part ###    
                # func 3 is related
                self.scrape_income_statement(year, season, headers, mar_newInfo_day_B, mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day)
                ### main part ###
            
                if (year != self.start_year) & (season == 1):
                    time.sleep(random.uniform(4, 9))
                    print('每逢第一季，休息N秒')
                    
        print('正常結束，無錯誤。 產出主鍵：年、季度。')
        
        self.PK = self.PK_year_season(self.dfs) # func 6 is related
        return self.dfs
    
    # func 2
    def get_newInfo_day(self, y, m, d):
        newInfo_day = datetime.datetime(y, m, d)
        if newInfo_day.weekday() == 5:  # Saturday
            newInfo_day += datetime.timedelta(days=2)
        elif newInfo_day.weekday() == 6:  # Sunday
            newInfo_day += datetime.timedelta(days=1)
        return newInfo_day
    
    # func 3
    def scrape_income_statement(self, year, season, headers, mar_newInfo_day_B, mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day):
        while True:
            C_or_A = 'C'
            something_wrong = False

            url = f'https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID={self.stock_id}&SYEAR={year}&SSEASON={season}&REPORT_ID=C'

            url_A = 'https://mops.twse.com.tw/server-java/t164sb01'
            payload = {
                'step': 1,
                'CO_ID': str(self.stock_id),
                'SYEAR': str(year),
                'SSEASON': str(season),
                'REPORT_ID': 'A'
            }

            try:
                req = urllib.request.Request(url, headers=headers)
                response = urllib.request.urlopen(req)
                html_content = response.read().decode('cp950')
                soup = BeautifulSoup(html_content, 'html.parser')
                tables = soup.find_all('table')

                ghost = tables[3]

                main_table = tables[1]
            except:
                try:
                    C_or_A = 'A'
                    res = requests.post(url_A, data=payload, headers=headers)
                    res.encoding = 'big5'
                    soup = BeautifulSoup(res.text, 'html.parser')
                    tables = soup.find_all('table')
                    if year >= 2019:
                        main_table = tables[1]
                    else:
                        main_table = tables[2]
                    
                except Exception as e:
                    something_wrong = True
                    print(f'於{year}Q{season}發生【錯誤1】:若於出現則自動休眠(16s)再重跑')
                    print(f'於{year}Q{season}發生{e}')
                    time.sleep(16)

            if something_wrong == False:
                if C_or_A == 'C':
                    print('---這是合併財報---')
                    try:
                        df = pd.read_html(StringIO(main_table))[0]
                        
                        income_statement_info_1season = [year, season, df]
                        # data_dict = df.process_df(df)
                        # self.dfs.append(data_dict)
                        self.dfs.append(df)
                        print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'finished')
                    except Exception as e:
                        something_wrong = True
                        print(f'於{year}Q{season}發生【錯誤2】:若於出現則自動休眠(16s)再重跑')
                        time.sleep(16)
                else:
                    print('---這是個別財報---')
                    try:
                        df = pd.read_html(StringIO(main_table))[0]
                        income_statement_info_1season = [year, season, df]
                        # data_dict = df.process_df(df)
                        # self.dfs.append(data_dict)
                        self.dfs.append(df)
                        print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'finished')
                    except Exception as e:
                        something_wrong = True
                        print(f'於{year}Q{season}發生【錯誤3】:若於出現則自動休眠(16s)再重跑')
                        time.sleep(16)

            time.sleep(random.uniform(0.5, 1.5))

            if something_wrong == False:
                break
    
    def process_df(self, df):
        data = df.transpose()
        data = data.reset_index().drop(columns='level_1')
        data.columns = data.iloc[0]
        data = data.drop(data.index[0])
        data = data.rename(columns={'會計項目':'Timestamp'})
        data = data.iloc[0]
        new_timestamp = data['Timestamp'].replace('年第','Q').replace('季','')
        data['Timestamp'] = new_timestamp
        index_to_terms_dict = {index: term for index, term in enumerate(data)}

        return index_to_terms_dict

    
                           
    def get_stock_EPS(self, stock_income_statement):
        # get feature EPS
        EPS_list = []

        for i, dfs in enumerate(stock_income_statement):
            if dfs[0] <= 2018:
                Q_EPS = dfs[2][dfs[2].iloc[:, 0].isin(['基本每股盈餘合計', '基本每股盈餘合計　Total basic earnings per share'])].iloc[:, 1].values[0]

                if dfs[1] == 4:
                    Q_EPS = Q_EPS - sum(EPS_list[-3:])
                    EPS_list.append(Q_EPS)
                else:
                    EPS_list.append(Q_EPS)
            else:
                Q_EPS = dfs[2][dfs[2].iloc[:, 1].isin(['基本每股盈餘合計', '基本每股盈餘合計　Total basic earnings per share'])].iloc[:, 2].values[0]

                if type(Q_EPS) == str:
                    if Q_EPS[0] == '(':
                        Q_EPS = float(Q_EPS[1:-1]) * (-1)
                    else:
                        Q_EPS = float(Q_EPS)

                if dfs[1] == 4:
                    Q_EPS = Q_EPS - sum(EPS_list[-3:])
                    EPS_list.append(Q_EPS)
                else:
                    EPS_list.append(Q_EPS)

        df_EPS = pd.DataFrame({'單季EPS': EPS_list})
        return df_EPS
    
    # func 5
    def get_rc_breaker(self, df, col_name='EPS創N季新高/低'):
        # get feature __ record breaker (創N季新高/低)
        N_ans = []

        for i in range(len(df)):
            if i > 0:
                value = df.iloc[i, 0]
                N = i + 1
                while N >= 2:
                    if value == df.iloc[0:i + 1, 0].tail(N).max():
                        N_ans.append(N)
                        break
                    elif value == df.iloc[0:i + 1, 0].tail(N).min():
                        N_ans.append(-N)
                        break
                    else:
                        N -= 1
            else:
                N_ans.append(i)
        df_rc_bk = pd.DataFrame({col_name: N_ans})

        return df_rc_bk
   # func 6
    def PK_year_season(self, dfs):
        y_list = [] # for year
        s_list = [] # for season
        
        for df in dfs:
            y_list.append(str(df[0]))
            s_list.append('Q' + str(df[1]))
        
        df_PK_year_season = pd.DataFrame({'年度': y_list, '季度': s_list})
        
        return df_PK_year_season

if __name__ == '__main__': 
    # example 1
    # IS_scraper = IncomeStatementsScraper(start_year=2021)
    # IS_3372 = IS_scraper.get_income_statements_hst(3372)
    
    # example 2
    # IS_scraper = IncomeStatementsScraper(end_year=2015)
    # IS_2337 = IS_scraper.get_income_statements_hst(2337)
    
    # example 3
    IS_scraper = IncomeStatementsScraper() #實際在抓歷史資料時，用默認值即可
    IS_3372 = IS_scraper.get_income_statements_hst(2330) # 3372 is a good example showing 2 kind of 財報（合併 or 個別）
    # EPS_3372 = IS_scraper.get_stock_EPS(IS_3372)                                              # 特徵之一，形式為df
    # EPS_rc_3372 = IS_scraper.get_rc_breaker(EPS_3372, '(自訂義名稱-創N期新高/低)')              # 特徵之二，形式為df
    
    #final_gift = pd.concat([IS_scraper.PK,EPS_3372,EPS_rc_3372], axis=1) # 把主鍵IS_scraper.PK與我們要的特徵合併起來
    # print('========================最終===============================')
    # print(final_gift) 
    # print('========================特徵1==============================')
    # print(EPS_3372)
    # print('========================特徵2==============================')
    # print(EPS_rc_3372)





2013年開爬!!
於2013Q1發生【錯誤1】:若於出現則自動休眠(16s)再重跑
於2013Q1發生list index out of range


KeyboardInterrupt: 

In [64]:
IS_2330

NameError: name 'IS_2330' is not defined

In [7]:
#!/usr/bin/env python
# coding: utf-8

import datetime
import random
import time
import urllib.request
import requests
from bs4 import BeautifulSoup
import pandas as pd

class Cash_Flow_Scraper:
    
    def __init__(self, start_year=2013, end_year=datetime.datetime.now().year):
        if start_year < 2013 or start_year < 0:
            raise ValueError("start_year 必須大於等於 2013 並且不能為負數")
        if end_year < start_year or end_year < 0:
            raise ValueError("end_year 必須大於等於 start_year 並且不能為負數")
        if end_year > datetime.datetime.now().year:
            print('end_year 強制設定為今年')
            self.end_year = datetime.datetime.now().year
            self.year_now = self.end_year
        else:
            self.end_year = end_year
            self.year_now = datetime.datetime.now().year
        
        self.start_year = start_year
        self.dfs = []
        
        self.year_range = list(range(self.start_year, self.end_year+1))
        self.season_range = list(range(1, 5))        
    
    # func 1 
    def get_cash_flow_hst(self, stock_id=2330): # need to choose a stock id
        self.stock_id = stock_id

        # Calculate the newInfo_days for 財報發佈日 
        # func 2 is related
        mar_newInfo_day_B = self.get_newInfo_day(1 + self.end_year, 3, 31)
        mar_newInfo_day = self.get_newInfo_day(self.end_year, 3, 31)
        may_newInfo_day = self.get_newInfo_day(self.end_year, 5, 15)
        aug_newInfo_day = self.get_newInfo_day(self.end_year, 8, 14)
        nov_newInfo_day = self.get_newInfo_day(self.end_year, 11, 14)

        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

        # Loop through each year and quarter to retrieve income statements
        for year in self.year_range:
            print(f'{year}年開爬!!')
            for season in self.season_range:                
                if year == self.end_year:
                    if (year == self.year_now - 1) & (season == 4):
                            if datetime.datetime.now() < mar_newInfo_day_B:
                                print('已爬至歷史最新資料，正常結束。 產出主鍵：年、季度。')
                                self.PK = self.PK_year_season(self.dfs) # func 6 is related
                                return self.dfs
                    
                    elif year == self.year_now:
                        checkTool = (mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day)
                        for i, newInfo_day in enumerate(checkTool):
                            if (datetime.datetime.now() < newInfo_day) & (season >= i):
                                print('已爬至歷史最新資料，正常結束。 產出主鍵：年、季度。')
                                self.PK = self.PK_year_season(self.dfs) # func 6 is related
                                return self.dfs
                ### main part ###    
                # func 3 is related
                self.scrape_cash_flow(year, season, headers, mar_newInfo_day_B, mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day)
                ### main part ###
            
                if (year != self.start_year) & (season == 1):
                    time.sleep(random.uniform(4, 9))
                    print('每逢第一季，休息N秒')
                    
        print('正常結束，無錯誤。 產出主鍵：年、季度。')
        
        self.PK = self.PK_year_season(self.dfs) # func 6 is related
        return self.dfs
    
    # func 2
    def get_newInfo_day(self, y, m, d):
        newInfo_day = datetime.datetime(y, m, d)
        if newInfo_day.weekday() == 5:  # Saturday
            newInfo_day += datetime.timedelta(days=2)
        elif newInfo_day.weekday() == 6:  # Sunday
            newInfo_day += datetime.timedelta(days=1)
        return newInfo_day
    
    # func 3
    def scrape_cash_flow(self, year, season, headers, mar_newInfo_day_B, mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day):
        while True:
            C_or_A = 'C'
            something_wrong = False

            url = f'https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID={self.stock_id}&SYEAR={year}&SSEASON={season}&REPORT_ID=C'

            url_A = 'https://mops.twse.com.tw/server-java/t164sb01'
            payload = {
                'step': 1,
                'CO_ID': str(self.stock_id),
                'SYEAR': str(year),
                'SSEASON': str(season),
                'REPORT_ID': 'A'
            }

            try:
                req = urllib.request.Request(url, headers=headers)
                response = urllib.request.urlopen(req)
                html_content = response.read().decode('cp950')
                soup = BeautifulSoup(html_content, 'html.parser')
                tables = soup.find_all('table')

                ghost = tables[3]

                main_table = tables[2]
            except:
                try:
                    C_or_A = 'A'
                    res = requests.post(url_A, data=payload, headers=headers)
                    res.encoding = 'big5'
                    soup = BeautifulSoup(res.text, 'html.parser')
                    tables = soup.find_all('table')
                    if year >= 2019:
                        main_table = tables[2]
                    else:
                        main_table = tables[3]
                except:
                    something_wrong = True
                    print(f'於{year}Q{season}發生【錯誤1】:若於出現則自動休眠(16s)再重跑')
                    time.sleep(16)

            if something_wrong == False:
                if C_or_A == 'C':
                    print('---這是合併財報---')
                    try:
                        df = pd.read_html(str(main_table))[0]
                        income_statement_info_1season = [year, season, df]
                        self.dfs.append(income_statement_info_1season)
                        print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'finished')
                    except Exception as e:
                        something_wrong = True
                        print(f'於{year}Q{season}發生【錯誤2】:若於出現則自動休眠(16s)再重跑')
                        time.sleep(16)
                else:
                    print('---這是個別財報---')
                    try:
                        df = pd.read_html(str(main_table))[0]
                        income_statement_info_1season = [year, season, df]
                        self.dfs.append(income_statement_info_1season)
                        print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'finished')
                    except Exception as e:
                        something_wrong = True
                        print(f'於{year}Q{season}發生【錯誤3】:若於出現則自動休眠(16s)再重跑')
                        time.sleep(16)

            time.sleep(random.uniform(0.5, 1.5))

            if something_wrong == False:
                break
    
    
    # func 6
    def PK_year_season(self, dfs):
        y_list = [] # for year
        s_list = [] # for season
        
        for df in dfs:
            y_list.append(str(df[0]))
            s_list.append('Q' + str(df[1]))
        
        df_PK_year_season = pd.DataFrame({'年度': y_list, '季度': s_list})
        
        return df_PK_year_season

if __name__ == '__main__': 
   
    

    BS_scraper = Balance_Sheet_Scraper(end_year=2015)
    result = BS_scraper.get_balance_sheet_hst(2337)
        

2013年開爬!!
---這是合併財報---
stock:2337 period:2013 Q1 finished
---這是合併財報---
stock:2337 period:2013 Q2 finished
---這是合併財報---
stock:2337 period:2013 Q3 finished
---這是合併財報---
stock:2337 period:2013 Q4 finished
2014年開爬!!
---這是合併財報---
stock:2337 period:2014 Q1 finished
每逢第一季，休息N秒
---這是合併財報---
stock:2337 period:2014 Q2 finished
---這是合併財報---
stock:2337 period:2014 Q3 finished
---這是合併財報---
stock:2337 period:2014 Q4 finished
2015年開爬!!
---這是合併財報---
stock:2337 period:2015 Q1 finished
每逢第一季，休息N秒
---這是合併財報---
stock:2337 period:2015 Q2 finished
---這是合併財報---
stock:2337 period:2015 Q3 finished
---這是合併財報---
stock:2337 period:2015 Q4 finished
正常結束，無錯誤。 產出主鍵：年、季度。


In [42]:
IS_3372

[]

In [12]:
#!/usr/bin/env python
# coding: utf-8

import datetime
import random
import time
import urllib.request
import requests
from bs4 import BeautifulSoup
import pandas as pd

class Balance_Sheet_Scraper:
    
    def __init__(self, start_year=2013, end_year=datetime.datetime.now().year):
        if start_year < 2013 or start_year < 0:
            raise ValueError("start_year 必須大於等於 2013 並且不能為負數")
        if end_year < start_year or end_year < 0:
            raise ValueError("end_year 必須大於等於 start_year 並且不能為負數")
        if end_year > datetime.datetime.now().year:
            print('end_year 強制設定為今年')
            self.end_year = datetime.datetime.now().year
            self.year_now = self.end_year
        else:
            self.end_year = end_year
            self.year_now = datetime.datetime.now().year
        
        self.start_year = start_year
        self.dfs = []
        
        self.year_range = list(range(self.start_year, self.end_year+1))
        self.season_range = list(range(1, 5))        
    
    # func 1 
    def get_balance_sheet_hst(self, stock_id=2330): # need to choose a stock id
        self.stock_id = stock_id

        # Calculate the newInfo_days for 財報發佈日 
        # func 2 is related
        mar_newInfo_day_B = self.get_newInfo_day(1 + self.end_year, 3, 31)
        mar_newInfo_day = self.get_newInfo_day(self.end_year, 3, 31)
        may_newInfo_day = self.get_newInfo_day(self.end_year, 5, 15)
        aug_newInfo_day = self.get_newInfo_day(self.end_year, 8, 14)
        nov_newInfo_day = self.get_newInfo_day(self.end_year, 11, 14)

        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

        # Loop through each year and quarter to retrieve income statements
        for year in self.year_range:
            print(f'{year}年開爬!!')
            for season in self.season_range:                
                if year == self.end_year:
                    if (year == self.year_now - 1) & (season == 4):
                            if datetime.datetime.now() < mar_newInfo_day_B:
                                print('已爬至歷史最新資料，正常結束。 產出主鍵：年、季度。')
                                self.PK = self.PK_year_season(self.dfs) # func 6 is related
                                return self.dfs
                    
                    elif year == self.year_now:
                        checkTool = (mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day)
                        for i, newInfo_day in enumerate(checkTool):
                            if (datetime.datetime.now() < newInfo_day) & (season >= i):
                                print('已爬至歷史最新資料，正常結束。 產出主鍵：年、季度。')
                                self.PK = self.PK_year_season(self.dfs) # func 6 is related
                                return self.dfs
                ### main part ###    
                # func 3 is related
                self.scrape_balance_sheet(year, season, headers, mar_newInfo_day_B, mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day)
                ### main part ###
            
                if (year != self.start_year) & (season == 1):
                    time.sleep(random.uniform(4, 9))
                    print('每逢第一季，休息N秒')
                    
        print('正常結束，無錯誤。 產出主鍵：年、季度。')
        
        self.PK = self.PK_year_season(self.dfs) # func 6 is related
        return self.dfs
    
    # func 2
    def get_newInfo_day(self, y, m, d):
        newInfo_day = datetime.datetime(y, m, d)
        if newInfo_day.weekday() == 5:  # Saturday
            newInfo_day += datetime.timedelta(days=2)
        elif newInfo_day.weekday() == 6:  # Sunday
            newInfo_day += datetime.timedelta(days=1)
        return newInfo_day
    
    # func 3
    def scrape_balance_sheet(self, year, season, headers, mar_newInfo_day_B, mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day):
        while True:
            C_or_A = 'C'
            something_wrong = False

            url = f'https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID={self.stock_id}&SYEAR={year}&SSEASON={season}&REPORT_ID=C'

            url_A = 'https://mops.twse.com.tw/server-java/t164sb01'
            payload = {
                'step': 1,
                'CO_ID': str(self.stock_id),
                'SYEAR': str(year),
                'SSEASON': str(season),
                'REPORT_ID': 'A'
            }

            try:
                req = urllib.request.Request(url, headers=headers)
                response = urllib.request.urlopen(req)
                html_content = response.read().decode('cp950')
                soup = BeautifulSoup(html_content, 'html.parser')
                tables = soup.find_all('table')

                ghost = tables[3]

                main_table = tables[3]
            except:
                try:
                    C_or_A = 'A'
                    res = requests.post(url_A, data=payload, headers=headers)
                    res.encoding = 'big5'
                    soup = BeautifulSoup(res.text, 'html.parser')
                    tables = soup.find_all('table')
                    if year >= 2019:
                        main_table = tables[3]
                    else:
                        main_table = tables[4]
                except:
                    something_wrong = True
                    print(f'於{year}Q{season}發生【錯誤1】:若於出現則自動休眠(16s)再重跑')
                    time.sleep(16)

            if something_wrong == False:
                if C_or_A == 'C':
                    print('---這是合併財報---')
                    try:
                        df = pd.read_html(str(main_table))[0]
                        income_statement_info_1season = [year, season, df]
                        self.dfs.append(income_statement_info_1season)
                        print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'finished')
                    except Exception as e:
                        something_wrong = True
                        print(f'於{year}Q{season}發生【錯誤2】:若於出現則自動休眠(16s)再重跑')
                        time.sleep(16)
                else:
                    print('---這是個別財報---')
                    try:
                        df = pd.read_html(str(main_table))[0]
                        income_statement_info_1season = [year, season, df]
                        self.dfs.append(income_statement_info_1season)
                        print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'finished')
                    except Exception as e:
                        something_wrong = True
                        print(f'於{year}Q{season}發生【錯誤3】:若於出現則自動休眠(16s)再重跑')
                        time.sleep(16)

            time.sleep(random.uniform(0.5, 1.5))

            if something_wrong == False:
                break
    
    
    # func 6
    def PK_year_season(self, dfs):
        y_list = [] # for year
        s_list = [] # for season
        
        for df in dfs:
            y_list.append(str(df[0]))
            s_list.append('Q' + str(df[1]))
        
        df_PK_year_season = pd.DataFrame({'年度': y_list, '季度': s_list})
        
        return df_PK_year_season

if __name__ == '__main__': 
   BS_scraper = Balance_Sheet_Scraper()
   result = BS_scraper.get_balance_sheet_hst()

        

2013年開爬!!
---這是合併財報---
stock:2330 period:2013 Q1 finished
---這是合併財報---
stock:2330 period:2013 Q2 finished
---這是合併財報---
stock:2330 period:2013 Q3 finished
---這是合併財報---
stock:2330 period:2013 Q4 finished
2014年開爬!!
---這是合併財報---
stock:2330 period:2014 Q1 finished
每逢第一季，休息N秒
---這是合併財報---
stock:2330 period:2014 Q2 finished
---這是合併財報---
stock:2330 period:2014 Q3 finished
---這是合併財報---
stock:2330 period:2014 Q4 finished
2015年開爬!!
---這是合併財報---
stock:2330 period:2015 Q1 finished
每逢第一季，休息N秒
---這是合併財報---
stock:2330 period:2015 Q2 finished
---這是合併財報---
stock:2330 period:2015 Q3 finished
---這是合併財報---
stock:2330 period:2015 Q4 finished
2016年開爬!!
---這是合併財報---
stock:2330 period:2016 Q1 finished
每逢第一季，休息N秒
---這是合併財報---
stock:2330 period:2016 Q2 finished
---這是合併財報---
stock:2330 period:2016 Q3 finished
---這是合併財報---
stock:2330 period:2016 Q4 finished
2017年開爬!!
---這是合併財報---
stock:2330 period:2017 Q1 finished
每逢第一季，休息N秒
---這是合併財報---
stock:2330 period:2017 Q2 finished
---這是合併財報---
stock:2330 period:2017 Q3 fin

In [19]:
from mongo import Mongo
import datetime as dt
class QuarterReportUpdater:
    def __init__(self, stock_id, start_year=2013, end_year=dt.datetime.now().year):
        # Initialize the scrapper
        self.scrapper = IncomeStatementsScraper(start_year, end_year)
        self.scrapper.stock_id = stock_id
        
        # Initialize the MongoDB connection
        self.mongo = Mongo(db='trading_bot', collection='quarter_report')
    
    def update_latest_data(self):
        # Get the latest data date from MongoDB
        latest_date = self.mongo.get_latest_data_date()
        
        # If there's no data in the database, start from the beginning
        if not latest_date:
            print("No data found in the database. Starting from the beginning.")
            start_date = datetime(self.scrapper.start_year, 1, 1)
        else:
            # Else start from the next quarter after the latest date
            next_quarter_month = (latest_date.month - 1) // 3 * 3 + 4
            if next_quarter_month <= 12:
                start_date = datetime(latest_date.year, next_quarter_month, 1)
            else:
                start_date = datetime(latest_date.year + 1, 1, 1)
                
        # Get the end date
        end_date = datetime(self.scrapper.end_year, 12, 31)
        
        # Scrape data for the given date range
        date_range = pd.date_range(start=start_date, end=end_date, freq='3M')
        for date in date_range:
            print(f"Scraping data for: {date}")
            balance_sheet = self.scrapper.get_balance_sheet(date)
            income_sheet = self.scrapper.get_income_sheet(date)
            cash_flow_sheet = self.scrapper.get_cash_flow_sheet(date)
            
            # Construct the record and send to MongoDB
            record = {
                'Timestamp': date,
                'stock_id': self.scrapper.stock_id,
                'balance_sheet': balance_sheet.to_dict(),
                'income_sheet': income_sheet.to_dict(),
                'cash_flow_sheet': cash_flow_sheet.to_dict()
            }
            self.mongo.send_document(record)
        print("Data update completed!")

In [ ]:
stock_id_list = Mongo(db='trading_bot',collection='quarter_report').get_stock_id_list()
stock_id_list

In [22]:
for id in stock_id_list:
    IncomeStatementUpdater(id)